# <center> Python Client Example
## <center> ENGR 580A2: Secure Vehicle and Industrial Networking
## <center> <img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="600" /> 
### <center> Instructor: Dr. Jeremy Daily<br>Fall 2021

# Web Clients
While often we think of a web client to be a browser, we can also use programs or libraries to access web based resources. 

Python has the requests library for these transactions.

https://requests.readthedocs.io/en/master/

Run this notebook after the Tornado server notebook is running.

In [1]:
import requests
import base64
import json

C:\Users\Jeremy\Anaconda3\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.10) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [32]:
url = "http://localhost:8080"
r = requests.get(url)
print(r)

<Response [200]>


What are the status codes?

https://developer.mozilla.org/en-US/docs/Web/HTTP/Status

200 is good.

In [8]:
#Responses have status codes
r.status_code

200

In [9]:
# Text base response
r.text

'{"message": "The current time is 1678329567.0037196"}'

In [5]:
# Bytes as the response. Use this to read binary (like photos)
r.content 

b'{"message": "The current time is 1678329522.615879"}'

In [10]:
# Access lower level bytes from the socket
r = requests.get(url, stream=True)
r.raw

In [11]:
r.raw.read()

b'{"message": "The current time is 1678329583.8718126"}'

In [12]:
# Headers are often not displayed to the client, but are available
# requests makes it a dictionary. 
# This is the header response from the server
r.headers

{'Server': 'TornadoServer/6.0.3', 'Content-Type': 'text/html; charset=UTF-8', 'Date': 'Thu, 09 Mar 2023 02:39:43 GMT', 'Etag': '"66ed23e11d7d3638e9904729a6f749b45b02aa75"', 'Content-Length': '53'}

In [15]:
r.encoding

'UTF-8'

In [16]:
r.url

'http://localhost:8080/'

In [17]:
#This is actualy json, so we can load it directly into a dictionary
r = requests.get(url)
print(r.status_code)
r.json()

200


{'message': 'The current time is 1678329773.4136622'}

In [33]:
plain_text = "I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character."
plain_text

'I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character.'

In [34]:
#Get the key (Don't actually do this)
r = requests.get(url+"/encrypted/")
print(r.status_code)
print(r.text)
pub_key_text = r.json()['pub']
print(pub_key_text)

200
{"pub": "-----BEGIN PUBLIC KEY-----\nMCowBQYDK2VuAyEA7n07LnIGILC8CPV0uHVftIztbQ7R2339QGpwS6vBGxI=\n-----END PUBLIC KEY-----\n"}
-----BEGIN PUBLIC KEY-----
MCowBQYDK2VuAyEA7n07LnIGILC8CPV0uHVftIztbQ7R2339QGpwS6vBGxI=
-----END PUBLIC KEY-----



In [35]:
from cryptography.hazmat.primitives.asymmetric.x25519 import X25519PrivateKey
from cryptography.hazmat.primitives import serialization
#Client needs to generate a key pair
private_key_for_client= X25519PrivateKey.generate()
private_key_for_client

In [36]:
client_public_pem = private_key_for_client.public_key().public_bytes(
       encoding=serialization.Encoding.PEM,
       format=serialization.PublicFormat.SubjectPublicKeyInfo
)
print(client_public_pem)

b'-----BEGIN PUBLIC KEY-----\nMCowBQYDK2VuAyEAHTXpOpyeVRTrB2GkC1YEXJTiScHLiTPsqQJohfwnClg=\n-----END PUBLIC KEY-----\n'


In [37]:
server_pub_key = serialization.load_pem_public_key(pub_key_text.encode('ascii'))
server_pub_key

In [38]:
shared_key = private_key_for_client.exchange(server_pub_key)
shared_key

b'U\\\xfb\x1b\xf7\xdb\x87b\xdc\x1b\xa4\xb0\xa2\x0b{\x85\x19@D\xc1\xee\x7f\\1\xb1\xe2F\xd5\x8b!\xe2\r'

In [39]:
# Send the server our public key
r = requests.post(url+"/encrypted/", json={
        'pub_key':client_public_pem.decode('utf-8'),
        'uid':1
    })
assert r.status_code == 200
r.json()

{'encrypted_session_key': 'gAAAAABkCUiZNiW3NDtbJ7PxvGr9Ay7Hxf4ZmaI4hu2J9gpDi4APZT7wJgZL3XtdK3onvoSVfDpBjqRWjDm1as1xmKbd2jnCLhSyMRo-PY-irBi1dtuClPpp7ABV2C-gF8NhDmN9dYts'}

In [43]:
enc_session_key = r.json()['encrypted_session_key']
enc_session_key

'gAAAAABkCUiZNiW3NDtbJ7PxvGr9Ay7Hxf4ZmaI4hu2J9gpDi4APZT7wJgZL3XtdK3onvoSVfDpBjqRWjDm1as1xmKbd2jnCLhSyMRo-PY-irBi1dtuClPpp7ABV2C-gF8NhDmN9dYts'

In [44]:
from cryptography.fernet import Fernet
f = Fernet(base64.urlsafe_b64encode(shared_key))
f

In [45]:
new_session_key = f.decrypt(enc_session_key.encode('utf-8'))
new_session_key

b'yg4arb3r3OcWI_ZyTgzRLC-gyFcNNTo1gygwSRkiWWg='

In [46]:
session_cipher = Fernet(new_session_key)
session_cipher

In [47]:
cipher_text = session_cipher.encrypt(plain_text.encode('utf-8'))
cipher_text

b'gAAAAABkCUjB3TfGNunRa4yRpg-mE5hBz-nmt26QgIgbjPqlS7ImW8VVRkG0C6HGtSY3FAZdRMCj27jZnaigAGpkaICoRFNiyRlJ5HLgruGGn4BfKhkdL7UChlEoJlCODn2JjS7jqSEUh742kVhkpWAvxFM_tcNEqR2bk5wfRMbDldsuIkLIOwFpxvRE0RVaaK5UJOIuWSA07NGYCKrEnWY7ABolWscJpOmutln2vlEwaz9Y6NqD99Gylx78sSlm_wBnc1ZFphYcv8fCBpvXVy5VxCYW75wWL3qbxvqjNI40v0bVVY2hLEo='

In [48]:
r = requests.post(url+"/encrypted/", 
                  json={'cipher_text':cipher_text.decode('utf-8'), })
print(r)
print(r.text)

<Response [200]>
{"decrypted message": "I have a dream that my four little children will one day live in a nation where they will not be judged by the color of their skin but by the content of their character."}


## Summary
* We can see how data can be encrypted and sent across the Internet. When using symmetric encryption, there needs to be a key exchange. This should never be done in the open, like we did in this example. 

* We introduced GET and POST

* We introduced JSON

* We talked about HTTP status codes

In [25]:
u"\U0001F44F"

'👏'